## Titanic FL

In [1]:
import pandas as pd
import numpy as np 
from preprocess import Preprocessor
import os

In [2]:
df = pd.read_csv('data/titanic.csv')

In [3]:
df.head()

,PassengerId,Survived,Pclass,Age,Sex,SibSp,Parch,Fare,Cabin,Embarked
0,1,0,3,22.0,male,1,0,7.2500,N,S
1,2,1,1,38.0,female,1,0,71.2833,C,C
2,3,1,3,26.0,female,0,0,7.9250,N,S
3,4,1,1,35.0,female,1,0,53.1000,C,S
4,5,0,3,35.0,male,0,0,8.0500,N,S


In [4]:
n_cols = len(df.columns)
df.columns = ['id'] + ['y'] + ['x' + str(i) for i in range(n_cols - 2)]
df.columns


Index(['id', 'y', 'x0', 'x1', 'x2', 'x3', 'x4', 'x5', 'x6', 'x7'], dtype='object')

In [5]:
processed_df = Preprocessor().fit_transform(df)
processed_df

,id,y,x0,x1,x2,x3,x4,x5,x6,x7
0,1,0,3,22.0,1,1,0,7.2500,7,2
1,2,1,1,38.0,0,1,0,71.2833,2,0
2,3,1,3,26.0,0,0,0,7.9250,7,2
3,4,1,1,35.0,0,1,0,53.1000,2,2
4,5,0,3,35.0,1,0,0,8.0500,7,2
...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,27.0,1,0,0,13.0000,7,2
887,888,1,1,19.0,0,0,0,30.0000,1,2
888,889,0,3,24.0,0,1,2,23.4500,7,2
889,890,1,1,26.0,1,0,0,30.0000,2,0


In [6]:
processed_df.to_csv("data/processed_titanic.csv")

In [7]:
num_data = processed_df.shape[0]
train_ratio = 0.2
train_df = processed_df[int(np.ceil(num_data * train_ratio)):]
valid_df = processed_df[:int(np.ceil(num_data * train_ratio))]

In [8]:
# train_df.to_csv('data/titanic_train.csv', index=False)
# valid_df.to_csv('data/titanic_valid.csv', index=False)

## Data split

In [9]:
# from tqdm import tqdm
# folders = ["8_0", "6_2", "4_4", "2_6", "0_8"]

# for folder in tqdm(folders):
    
#     if not os.path.exists(os.path.join("data", folder)):
#         os.mkdir(os.path.join("data", folder))
        
#     pos = int(folder.split("_")[0])
#     train_guest_df = train_df[train_df.columns[: pos + 2]]
#     train_host_df = train_df[['id'] + list(train_df.columns[pos + 2:])]
#     train_n_cols = len(train_host_df.columns)
#     train_host_df.columns = ['id'] + ['x' + str(i) for i in range(train_n_cols - 1)]
    
#     valid_guest_df = valid_df[valid_df.columns[: pos + 2]]
#     valid_host_df = valid_df[['id'] + list(valid_df.columns[pos + 2:])]
#     valid_n_cols = len(valid_host_df.columns)
#     valid_host_df.columns = ['id'] + ['x' + str(i) for i in range(valid_n_cols - 1)]

#     train_guest_df.to_csv(os.path.join("data", folder, 'train_guest.csv'), index=False)
#     train_host_df.to_csv(os.path.join("data", folder, 'train_host.csv'), index=False)
#     valid_guest_df.to_csv(os.path.join("data", folder, 'valid_guest.csv'), index=False)
#     valid_host_df.to_csv(os.path.join("data", folder, 'valid_host.csv'), index=False)
    
    

In [10]:
import os

In [11]:
!pipeline --help

Usage: pipeline [OPTIONS] COMMAND [ARGS]...

Options:
  --help  Show this message and exit.

Commands:
  config  pipeline config tool
  init     - DESCRIPTION: Pipeline Config Command.


Assume we have a `FATE Flow Service` in 127.0.0.1:9380(defaults in standalone), then exec

In [12]:
!pipeline init --ip 127.0.0.1 --port 9380

Pipeline configuration succeeded.


In [13]:
pwd

'/data/projects/fate/persistence/titanic-fl'

### Upload data

In [14]:
import os

In [15]:
!pipeline --help

Usage: pipeline [OPTIONS] COMMAND [ARGS]...

Options:
  --help  Show this message and exit.

Commands:
  config  pipeline config tool
  init     - DESCRIPTION: Pipeline Config Command.


Assume we have a `FATE Flow Service` which name is `fateflow` and the port is `9380`, then exec

In [16]:
!pipeline init --ip fateflow --port 9380

Pipeline configuration succeeded.


 Before start a modeling task, the data to be used should be uploaded. 
 Typically, a party is usually a cluster which include multiple nodes. Thus, when we upload these data, the data will be allocated to those nodes.

In [17]:
from pipeline.backend.pipeline import PipeLine

/data/projects/python/venv/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Make a `pipeline` instance:

    - initiator: 
        * role: guest
        * party: 9999
    - roles:
        * guest: 9999

note that only local party id is needed.
    

In [18]:
pipeline_upload = PipeLine().set_initiator(role='guest', party_id=9999).set_roles(guest=9999)

Define partitions for data storage

In [19]:
partition = 4

Define table name and namespace, which will be used in FATE job configuration

In [20]:
# dense_data_guest = {"name": "titanic_guest", "namespace": f"experiment"}
# dense_data_host = {"name": "titanic_host", "namespace": f"experiment"}

dense_data_train_guest = {"name": "titanic_train_guest", "namespace": f"experiment"}
dense_data_train_host = {"name": "titanic_train_host", "namespace": f"experiment"}
dense_data_valid_guest = {"name": "titanic_valid_guest", "namespace": f"experiment"}
dense_data_valid_host  = {"name": "titanic_valid_host", "namespace": f"experiment"}

Now, we add data to be uploaded

In [21]:
folder = "2_6"

data_base = "/data/projects/fate/persistence/titanic-fl/data"
pipeline_upload.add_upload_data(file=os.path.join(data_base, folder, "train_guest.csv"),
                                table_name=dense_data_train_guest["name"],             # table name
                                namespace=dense_data_train_guest["namespace"],         # namespace
                                head=1, partition=partition)               # data info

pipeline_upload.add_upload_data(file=os.path.join(data_base, folder, "train_host.csv"),
                                table_name=dense_data_train_host["name"],
                                namespace=dense_data_train_host["namespace"],
                                head=1, partition=partition)

pipeline_upload.add_upload_data(file=os.path.join(data_base, folder, "valid_guest.csv"),
                                table_name=dense_data_valid_guest["name"],             # table name
                                namespace=dense_data_valid_guest["namespace"],         # namespace
                                head=1, partition=partition)               # data info

pipeline_upload.add_upload_data(file=os.path.join(data_base, folder, "valid_host.csv"),
                                table_name=dense_data_valid_host["name"],
                                namespace=dense_data_valid_host["namespace"],
                                head=1, partition=partition)

We can then upload data

In [22]:
pipeline_upload.upload(drop=1)

 UPLOADING:||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||100.00%


2023-07-27 01:14:49.075 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:83 - Job id is 202307270114488084200

2023-07-27 01:14:49.082 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:98 - Job is still waiting, time elapse: 0:00:00
2023-07-27 01:14:50.091 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:98 - Job is still waiting, time elapse: 0:00:01
m2023-07-27 01:14:51.106 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:125 - 
2023-07-27 01:14:51.107 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component upload_0, time elapse: 0:00:02
2023-07-27 01:14:52.121 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component upload_0, time elapse: 0:00:03
2023-07-27 01:14:53.137 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component upload_0, time elapse: 0:00:04
2023-07-27 01:14:54.150 | INFO     | pipel

 UPLOADING:||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||100.00%

2023-07-27 01:14:59.386 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:83 - Job id is 202307270114592449440

2023-07-27 01:14:59.392 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:98 - Job is still waiting, time elapse: 0:00:00


2023-07-27 01:15:00.400 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:98 - Job is still waiting, time elapse: 0:00:01
m2023-07-27 01:15:01.416 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:125 - 
2023-07-27 01:15:01.417 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component upload_0, time elapse: 0:00:02
2023-07-27 01:15:02.432 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component upload_0, time elapse: 0:00:03
2023-07-27 01:15:03.446 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component upload_0, time elapse: 0:00:04
2023-07-27 01:15:04.460 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component upload_0, time elapse: 0:00:05
2023-07-27 01:15:05.484 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component upload_0, time elapse: 0:00:06
2023-07-27 01:15:0

...

2023-07-27 01:15:09.712 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:83 - Job id is 202307270115095404410

2023-07-27 01:15:09.719 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:98 - Job is still waiting, time elapse: 0:00:00


2023-07-27 01:15:10.728 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:98 - Job is still waiting, time elapse: 0:00:01
m2023-07-27 01:15:12.757 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:125 - 
2023-07-27 01:15:12.758 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component upload_0, time elapse: 0:00:03
2023-07-27 01:15:13.773 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component upload_0, time elapse: 0:00:04
2023-07-27 01:15:14.788 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component upload_0, time elapse: 0:00:05
2023-07-27 01:15:15.801 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component upload_0, time elapse: 0:00:06
2023-07-27 01:15:16.816 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component upload_0, time elapse: 0:00:07
2023-07-27 01:15:1

 UPLOADING:||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||100.00%

2023-07-27 01:15:21.015 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:83 - Job id is 202307270115208731650

2023-07-27 01:15:21.022 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:98 - Job is still waiting, time elapse: 0:00:00


2023-07-27 01:15:22.031 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:98 - Job is still waiting, time elapse: 0:00:01
m2023-07-27 01:15:23.046 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:125 - 
2023-07-27 01:15:23.047 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component upload_0, time elapse: 0:00:02
2023-07-27 01:15:24.061 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component upload_0, time elapse: 0:00:03
2023-07-27 01:15:25.074 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component upload_0, time elapse: 0:00:04
2023-07-27 01:15:26.089 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component upload_0, time elapse: 0:00:05
2023-07-27 01:15:27.102 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component upload_0, time elapse: 0:00:06
2023-07-27 01:15:2

## Modeling

In [23]:
import argparse
from sklearn.metrics import accuracy_score

import torch as t
from torch import nn

from pipeline import fate_torch_hook
from pipeline.backend.pipeline import PipeLine
from pipeline.component import DataTransform
from pipeline.component import HeteroSecureBoost, HeteroLR, HeteroNN
from pipeline.component import Intersection
from pipeline.component import Reader
from pipeline.interface import Data
from pipeline.component import Evaluation
from pipeline.interface import Model

from pipeline.utils.tools import load_job_config

In [24]:
guest, host, arbiter = 9999, 10000, 10000

# init pipeline
pipeline = PipeLine().set_initiator(role="guest", party_id=guest).set_roles(guest=guest, host=host, arbiter=arbiter)

In [25]:
# set data reader and data-io
reader_0, reader_1 = Reader(name="reader_0"), Reader(name="reader_1")
reader_0.get_party_instance(role="guest", party_id=guest).component_param(table=dense_data_train_guest)
reader_0.get_party_instance(role="host", party_id=host).component_param(table=dense_data_train_host)
# reader_1.get_party_instance(role="guest", party_id=guest).component_param(table=dense_data_valid_guest)
# reader_1.get_party_instance(role="host", party_id=host).component_param(table=dense_data_valid_host)

In [26]:
# Get the component 

# guest, host = 9999, 10000

# # init pipeline
# pipeline = PipeLine().set_initiator(role="guest", party_id=guest).set_roles(guest=guest, host=host,)
# pipeline.add_component(reader_0)
# pipeline.compile()
# pipeline.fit()

In [27]:
data_transform_0, data_transform_1 = DataTransform(name="data_transform_0"), DataTransform(name="data_transform_1")

data_transform_0.get_party_instance(
    role="guest", party_id=guest).component_param(
    with_label=True, output_format="dense")
data_transform_0.get_party_instance(role="host", party_id=host).component_param(with_label=False)
# data_transform_1.get_party_instance(
#     role="guest", party_id=guest).component_param(
#     with_label=True, output_format="dense")
# data_transform_1.get_party_instance(role="host", party_id=host).component_param(with_label=False)

In [28]:
# data intersect component
intersect_0 = Intersection(name="intersection_0")
# intersect_1 = Intersection(name="intersection_1")

pipeline.add_component(reader_0)
pipeline.add_component(data_transform_0, data=Data(data=reader_0.output.data))
pipeline.add_component(intersect_0, data=Data(data=data_transform_0.output.data))

### HeteroSecureBoost

In [29]:
# secure boost component
hetero_secureboost_0 = HeteroSecureBoost(name="hetero_secureboost_0",
                                         num_trees=5,
                                         bin_num=16,
                                         task_type="classification",
                                         objective_param={"objective": "cross_entropy"},
                                         encrypt_param={"method": "iterativeAffine"},
                                         tree_param={"max_depth": 3})

pipeline.add_component(hetero_secureboost_0, data=Data(train_data=intersect_0.output.data))

 # evaluation component
evaluation_0 = Evaluation(name="evaluation_0", eval_type="binary")
pipeline.add_component(evaluation_0, data=Data(data=hetero_secureboost_0.output.data))
pipeline.compile()
pipeline.fit()

2023-07-27 01:15:31.778 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:83 - Job id is 202307270115312178890

2023-07-27 01:15:31.785 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:98 - Job is still waiting, time elapse: 0:00:00
2023-07-27 01:15:32.795 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:98 - Job is still waiting, time elapse: 0:00:01
m2023-07-27 01:15:33.812 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:125 - 
2023-07-27 01:15:33.814 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component reader_0, time elapse: 0:00:02
2023-07-27 01:15:34.830 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component reader_0, time elapse: 0:00:03
2023-07-27 01:15:35.856 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component reader_0, time elapse: 0:00:04
2023-07-27 01:15:36.872 | INFO     | pipel

2023-07-27 01:16:08.425 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component data_transform_0, time elapse: 0:00:36
m2023-07-27 01:16:09.439 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:125 - 
2023-07-27 01:16:09.441 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component intersection_0, time elapse: 0:00:37
2023-07-27 01:16:10.455 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component intersection_0, time elapse: 0:00:38
2023-07-27 01:16:11.471 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component intersection_0, time elapse: 0:00:39
2023-07-27 01:16:12.485 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component intersection_0, time elapse: 0:00:40
2023-07-27 01:16:13.501 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component intersecti

2023-07-27 01:16:46.037 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_secureboost_0, time elapse: 0:01:14
2023-07-27 01:16:47.053 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_secureboost_0, time elapse: 0:01:15
2023-07-27 01:16:48.068 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_secureboost_0, time elapse: 0:01:16
2023-07-27 01:16:49.083 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_secureboost_0, time elapse: 0:01:17
2023-07-27 01:16:50.107 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_secureboost_0, time elapse: 0:01:18
2023-07-27 01:16:51.122 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_secureboost_0, time elapse: 0:01:19
2023-07-27 01:16:52.137 | INFO     | pip

2023-07-27 01:17:23.662 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_secureboost_0, time elapse: 0:01:51
2023-07-27 01:17:24.677 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_secureboost_0, time elapse: 0:01:52
2023-07-27 01:17:25.691 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_secureboost_0, time elapse: 0:01:53
2023-07-27 01:17:26.708 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_secureboost_0, time elapse: 0:01:54
2023-07-27 01:17:27.722 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_secureboost_0, time elapse: 0:01:55
2023-07-27 01:17:28.738 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_secureboost_0, time elapse: 0:01:56
2023-07-27 01:17:29.753 | INFO     | pip

2023-07-27 01:18:01.217 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_secureboost_0, time elapse: 0:02:29
2023-07-27 01:18:02.230 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_secureboost_0, time elapse: 0:02:30
2023-07-27 01:18:03.246 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_secureboost_0, time elapse: 0:02:31
2023-07-27 01:18:04.260 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_secureboost_0, time elapse: 0:02:32
2023-07-27 01:18:05.274 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_secureboost_0, time elapse: 0:02:33
2023-07-27 01:18:06.289 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_secureboost_0, time elapse: 0:02:34
2023-07-27 01:18:07.303 | INFO     | pip

...

2023-07-27 01:23:06.900 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_secureboost_0, time elapse: 0:07:35
2023-07-27 01:23:07.916 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_secureboost_0, time elapse: 0:07:36
2023-07-27 01:23:08.931 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_secureboost_0, time elapse: 0:07:37
2023-07-27 01:23:09.946 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_secureboost_0, time elapse: 0:07:38
2023-07-27 01:23:10.962 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_secureboost_0, time elapse: 0:07:39
2023-07-27 01:23:11.977 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_secureboost_0, time elapse: 0:07:40
2023-07-27 01:23:12.992 | INFO     | pip

2023-07-27 01:23:44.463 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_secureboost_0, time elapse: 0:08:12
2023-07-27 01:23:45.478 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_secureboost_0, time elapse: 0:08:13
2023-07-27 01:23:46.493 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_secureboost_0, time elapse: 0:08:14
2023-07-27 01:23:47.507 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_secureboost_0, time elapse: 0:08:15
2023-07-27 01:23:48.521 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_secureboost_0, time elapse: 0:08:16
2023-07-27 01:23:49.536 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_secureboost_0, time elapse: 0:08:17
2023-07-27 01:23:50.556 | INFO     | pip

2023-07-27 01:24:22.044 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_secureboost_0, time elapse: 0:08:50
2023-07-27 01:24:23.058 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_secureboost_0, time elapse: 0:08:51
2023-07-27 01:24:24.074 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_secureboost_0, time elapse: 0:08:52
2023-07-27 01:24:25.088 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_secureboost_0, time elapse: 0:08:53
2023-07-27 01:24:26.102 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_secureboost_0, time elapse: 0:08:54
2023-07-27 01:24:27.117 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_secureboost_0, time elapse: 0:08:55
2023-07-27 01:24:28.132 | INFO     | pip

2023-07-27 01:24:59.599 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_secureboost_0, time elapse: 0:09:27
2023-07-27 01:25:00.614 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_secureboost_0, time elapse: 0:09:28
2023-07-27 01:25:01.629 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_secureboost_0, time elapse: 0:09:29
2023-07-27 01:25:02.644 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_secureboost_0, time elapse: 0:09:30
2023-07-27 01:25:03.658 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_secureboost_0, time elapse: 0:09:31
2023-07-27 01:25:04.672 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_secureboost_0, time elapse: 0:09:32
2023-07-27 01:25:05.687 | INFO     | pip

2023-07-27 01:25:37.153 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_secureboost_0, time elapse: 0:10:05
2023-07-27 01:25:38.167 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_secureboost_0, time elapse: 0:10:06
2023-07-27 01:25:39.182 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_secureboost_0, time elapse: 0:10:07
2023-07-27 01:25:40.198 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_secureboost_0, time elapse: 0:10:08
2023-07-27 01:25:41.216 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_secureboost_0, time elapse: 0:10:09
2023-07-27 01:25:42.232 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_secureboost_0, time elapse: 0:10:10
2023-07-27 01:25:43.247 | INFO     | pip

## Evaluation

In [31]:
print("fitting hetero secureboost done, result:")
print(pipeline.get_component("hetero_secureboost_0").get_summary())

print('start to predict')

# predict
# deploy required components
pipeline.deploy_component([data_transform_0, intersect_0, hetero_secureboost_0, evaluation_0])

predict_pipeline = PipeLine()
# add data reader onto predict pipeline
predict_pipeline.add_component(reader_0)
# add selected components from train pipeline onto predict pipeline
# specify data source
predict_pipeline.add_component(
    pipeline, data=Data(
        predict_input={
            pipeline.data_transform_0.input.data: reader_0.output.data}))

# run predict model
predict_pipeline.predict()
predict_result = predict_pipeline.get_component("hetero_secureboost_0").get_output_data()

fitting hetero secureboost done, result:
{'best_iteration': -1, 'feature_importance': {'host_10000_x0': 5, 'host_10000_x1': 2, 'host_10000_x3': 8, 'host_10000_x4': 2, 'host_10000_x5': 3, 'x0': 7, 'x1': 8}, 'is_converged': False, 'loss_history': [0.5683997712586908, 0.5004904863028677, 0.46123841376614505, 0.4297120786032048, 0.4130773674230723], 'validation_metrics': None}
start to predict


2023-07-27 01:26:06.490 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:83 - Job id is 202307270126058526180

2023-07-27 01:26:06.498 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:98 - Job is still waiting, time elapse: 0:00:00
2023-07-27 01:26:07.506 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:98 - Job is still waiting, time elapse: 0:00:01
m2023-07-27 01:26:09.535 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:125 - 
2023-07-27 01:26:09.536 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component reader_0, time elapse: 0:00:03
2023-07-27 01:26:10.555 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component reader_0, time elapse: 0:00:04
2023-07-27 01:26:11.575 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component reader_0, time elapse: 0:00:05
2023-07-27 01:26:12.593 | INFO     | pipel

2023-07-27 01:26:44.156 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component data_transform_0, time elapse: 0:00:37
m2023-07-27 01:26:45.170 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:125 - 
2023-07-27 01:26:45.172 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component intersection_0, time elapse: 0:00:38
2023-07-27 01:26:46.186 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component intersection_0, time elapse: 0:00:39
2023-07-27 01:26:47.202 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component intersection_0, time elapse: 0:00:40
2023-07-27 01:26:48.217 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component intersection_0, time elapse: 0:00:41
2023-07-27 01:26:49.231 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component intersecti

2023-07-27 01:27:21.787 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_secureboost_0, time elapse: 0:01:15
2023-07-27 01:27:22.802 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_secureboost_0, time elapse: 0:01:16
2023-07-27 01:27:23.817 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_secureboost_0, time elapse: 0:01:17
2023-07-27 01:27:24.832 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_secureboost_0, time elapse: 0:01:18
2023-07-27 01:27:25.847 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_secureboost_0, time elapse: 0:01:19
2023-07-27 01:27:26.861 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_secureboost_0, time elapse: 0:01:20
2023-07-27 01:27:27.877 | INFO     | pip

2023-07-27 01:27:59.380 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_secureboost_0, time elapse: 0:01:52
2023-07-27 01:28:00.395 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_secureboost_0, time elapse: 0:01:53
2023-07-27 01:28:01.412 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_secureboost_0, time elapse: 0:01:54
2023-07-27 01:28:02.434 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_secureboost_0, time elapse: 0:01:55
2023-07-27 01:28:03.448 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_secureboost_0, time elapse: 0:01:56
2023-07-27 01:28:04.463 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_secureboost_0, time elapse: 0:01:57
2023-07-27 01:28:05.482 | INFO     | pip

2023-07-27 01:28:36.977 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component evaluation_0, time elapse: 0:02:30
2023-07-27 01:28:37.992 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component evaluation_0, time elapse: 0:02:31
2023-07-27 01:28:39.008 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component evaluation_0, time elapse: 0:02:32
2023-07-27 01:28:40.025 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component evaluation_0, time elapse: 0:02:33
2023-07-27 01:28:41.041 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component evaluation_0, time elapse: 0:02:34
2023-07-27 01:28:42.056 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component evaluation_0, time elapse: 0:02:35
2023-07-27 01:28:43.070 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_st

In [32]:
predict_result.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 712 entries, 0 to 711
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   id              712 non-null    object
 1   label           712 non-null    object
 2   predict_result  712 non-null    object
 3   predict_score   712 non-null    object
 4   predict_detail  712 non-null    object
 5   type            712 non-null    object
dtypes: object(6)
memory usage: 33.5+ KB


In [33]:
predict_result.head()

,id,label,predict_result,predict_score,predict_detail,type
0,509,0,0,0.21000633144840763,"{'0': 0.7899936685515924, '1': 0.2100063314484...",predict
1,467,0,0,0.1707587114621902,"{'0': 0.8292412885378098, '1': 0.1707587114621...",predict
2,390,1,1,0.848061900537295,"{'0': 0.15193809946270498, '1': 0.848061900537...",predict
3,301,1,1,0.5869551245970187,"{'0': 0.4130448754029813, '1': 0.5869551245970...",predict
4,745,1,0,0.21000633144840763,"{'0': 0.7899936685515924, '1': 0.2100063314484...",predict


In [34]:
predict_result['predict_result']

0      0
1      0
2      1
3      1
4      0
      ..
707    0
708    0
709    0
710    0
711    0
Name: predict_result, Length: 712, dtype: object

In [35]:
pipeline.get_component("hetero_secureboost_0").get_summary()

{'best_iteration': -1,
 'feature_importance': {'host_10000_x0': 5,
  'host_10000_x1': 2,
  'host_10000_x3': 8,
  'host_10000_x4': 2,
  'host_10000_x5': 3,
  'x0': 7,
  'x1': 8},
 'is_converged': False,
 'loss_history': [0.5683997712586908,
  0.5004904863028677,
  0.46123841376614505,
  0.4297120786032048,
  0.4130773674230723],
 'validation_metrics': None}

In [36]:
## Calculate Accuracy
y_true, y_pred = predict_result['label'], predict_result['predict_result']
acc = accuracy_score(y_true, y_pred)
print("accuary: ", acc)

accuary:  0.8328651685393258


In [43]:
from sklearn import metrics

threshold = 0.5

# 將機率值轉換成二元的 0 和 1
fpr, tpr, thresholds = metrics.roc_curve(y_true.values.astype(int), y_pred.values.astype(int))
metrics.auc(fpr, tpr)

0.8023795991993872